In [1]:
# CONFIGURAÇÕES INICIAIS DO NOTEBOOK
import os
import re

# Importando bibliotecas para análise de dados
import pandas as pd

from myutil import instance_problems_filenames

In [2]:
# Diretório contendo os dados gerados pela execução da simulação
DATA_FOLDER = os.path.join("..", "outputdata", "heuristics")

os.path.exists(DATA_FOLDER)

True

In [3]:
dfInstanceProblems = pd.read_csv(os.path.join(DATA_FOLDER,"instance_problem.csv"))

dfInstanceProblems.set_index("Problem", inplace=True)

dfInstanceProblems.head()

,Vertices,Edges,Terminals,Global Optimum
Problem,,,,
B1,50,63,9,82
B2,50,63,13,83
B3,50,63,25,138
B4,50,100,9,59
B5,50,100,13,61


In [4]:
len(dfInstanceProblems.index)

18

In [5]:
dfInstanceProblems.loc["B10","Global Optimum"]

86

## Leitura dos dados das simulações

Os arquivos se encontram no diretório descrito pela variável DATA_FOLDER.

A nomeação dos arquivos segue o seguinte padrão:

    [SIGLA_HEURISTICA]_[NOME_INSTÂNCIA_PROBLEMA].CSV
    
As siglas para heurística são:

    - PMH - Prunning MST Heuristic
    - SPH - Shortest Path Heuristic
    - SPHPrim - Shortest Path Heuristic with Prim
    - SPHO - Shortest Path Heuristic with Origin

Os nomes dos arquivos para cada instância do problema são aqueles descritos na OR-Library. 
A geração desses nomes é facilitada pela função [instance_problems_filenames] no módulo [myutil.py] desse diretório [notebooks].

In [6]:
def read_heuristic_data():
    regex = re.compile("stein(\w\d+).csv")

    HEURISTICAS = ["PMH", "SPH", "SPHPrim", "SPHO"]

    data_picies = list()
    for heuristica in HEURISTICAS:
        for dataset in instance_problems_filenames("b", file_extension="csv"):
            
            filename = os.path.join(DATA_FOLDER,f'{heuristica}_{dataset}')
            instance_problem = regex.findall(dataset)[0].upper()

            df = pd.read_csv(filename)
            df['Problem'] = instance_problem
            data_picies.append(df)
        
    return pd.concat(data_picies, ignore_index=True)

In [7]:
%%time
df_data = read_heuristic_data()

Wall time: 925 ms


In [8]:
df_data = df_data.merge(dfInstanceProblems["Global Optimum"], right_index=True, left_on="Problem")\
       .sort_values(by=["Heuristic","Problem", "Node"])\
       .reindex(['Heuristic', 'Problem', 'Node', 'Is terminal', 'Cost', 'Global Optimum'],
                axis='columns')

In [9]:
df_data["Success"] = df_data["Cost"] == df_data["Global Optimum"]

In [10]:
df_data.head()

,Heuristic,Problem,Node,Is terminal,Cost,Global Optimum,Success
42,PMH,B1,1,False,88,82,False
0,PMH,B1,2,False,89,82,False
9,PMH,B1,3,False,99,82,False
4,PMH,B1,4,False,89,82,False
5,PMH,B1,5,False,89,82,False


In [11]:
DATA_FOLDER

'..\\outputdata\\heuristics'

In [12]:
df_data.to_csv(os.path.join(DATA_FOLDER,'results.csv'),index=False)